# Setup

## Load Packages

In [1]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import openai
import re
import time
from scripts.prepare_data_helpers import prepare_examples, create_query
import spacy
import textacy
from dotenv import load_dotenv

load_dotenv()
nlp = spacy.load('en_core_web_sm')

/Users/marcbraun/.conda/envs/732A81-Text_Mining/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prepare Data

In [2]:
train1 = pd.read_csv('../../e-SNLI/dataset/esnli_train_1.csv')
train2 = pd.read_csv('../../e-SNLI/dataset/esnli_train_2.csv')
train = pd.concat([train1, train2])
dev = pd.read_csv('../../e-SNLI/dataset/esnli_dev.csv')
test = pd.read_csv('../../e-SNLI/dataset/esnli_test.csv')

train = train.dropna(subset=['Sentence1', 'Sentence2', 'Explanation_1'])
train = train.reset_index(drop=True)

np.random.seed(12345) # seed for numpy package
test_indices = list(np.random.choice(test.index.values, size=1000, replace=False))
test = test.loc[test_indices]
test = test.reset_index(drop=True)

In [3]:
gpt_pred = pd.read_csv('../output_data/test_predictions.csv', sep=';')

In [4]:
gpt_pred

,Unnamed: 0,pairID,gold_label,Sentence1,Sentence2,Explanation_1,Sentence1_marked_1,Sentence2_marked_1,Sentence1_Highlighted_1,Sentence2_Highlighted_1,...,Sentence1_Highlighted_2,Sentence2_Highlighted_2,Explanation_3,Sentence1_marked_3,Sentence2_marked_3,Sentence1_Highlighted_3,Sentence2_Highlighted_3,prompt,pred_explanation,pred_label
0,0,4813951931.jpg#3r1e,entailment,A young man in an orange hat walks his bicycle...,A man in an orange hat walks his bike down a s...,"A bicycle and a bike, are the same object.",A young man in an orange hat walks his *bicyc...,A man in an orange hat walks his *bike* down ...,9,8,...,"9,1,2","8,1",The young man in an orange hart is the man wal...,A *young* *man* in *an* *orange* *hat* *walks...,A *man* in *an* *orange* *hat* *walks* *his* ...,"1,2,4,5,6,7,8,9,10,12","1,5,4,3,11,9,8,7,6",Statement: A man in a suit and dirty dress-sho...,The man is young and the hat is orange.,entailment
1,1,4678320536.jpg#1r1c,contradiction,A woman standing with smile on her face and sh...,A woman stands holding her baby and crying.,If a woman has a smile on her face then she is...,A woman standing with *smile* *on* *her* *face...,A woman stands holding her baby and *crying.*,"4,5,6,7",7,...,13,5,the woman is either crying or has a smile,A *woman* standing with *smile* on her face an...,A *woman* stands holding her baby and *crying.*,"4,1","1,7",Statement: a white dog runs across the water.\...,"The woman is smiling and holding a wood, not a...",contradiction
2,2,661749711.jpg#2r1e,entailment,A dog jumps to catch a red ball outside.,the dog is jumping.,Jumping is an action needed to catch a ball.,A *dog* *jumps* to *catch* a red *ball* *outs...,the *dog* is *jumping.*,"2,7,8,1,4","3,1",...,"1,2","1,3","If the dog jumps to catch, the dog is jumping.",A dog *jumps* to *catch* a red ball outside.,the dog is *jumping.*,"2,4",3,Statement: A young child rides a bike with a b...,The statement is repeating the same information.,tautology
3,3,44904567.jpg#1r1c,contradiction,A man wielding an electric razor is gleefully ...,There is only one person present.,There cannot be one person present if one man ...,A *man* wielding an electric razor is gleefull...,There is only *one* *person* *present.*,"1,10,11","3,4,5",...,10,"3,2",It cannot be claimed that there is only one pe...,A *man* wielding an electric razor is gleefull...,There is *only* *one* *person* present.,"1,10,11","2,3,4",Statement: A young boy wearing safety glasses ...,There are two people present in the statement.,contradiction
4,4,32542645.jpg#0r1n,neutral,A group of people stares at a wall that is fil...,A group of people analyze art together,Not necessary that they analyze the art.,A group of people stares at a wall that is fil...,A group of people *analyze* art together,{},4,...,{},"4,5",The group may have just been admiring as oppos...,A group of people stares at a wall that is fil...,A group of people *analyze* art *together*,{},"4,6",Statement: Young mother and child heading some...,Staring at a wall filled with drawings is a fo...,entailment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,2429284131.jpg#2r1c,contradiction,Three men are standing on stage performing.,Women are playing trumpets on the street.,"either there are men on stage, or women playin...",Three *men* are standing on stage performing.,*Women* are playing trumpets on the street.,1,0,...,1,0,There can be either men or women.,Three *men* are standing on stage performing.,*Women* are playing trumpets on the street.,1,0,Statement: Several basketball players are grab...,"The men are standing on stage performing, whil...",contradiction
996,996,4460943467.jpg#0r1e,entailment,3 young man in hoods standing in the middle of...,Three hood wearing people stand in a street.,3 Man in hoods are hood wearing people.,*3* young *man* in *hoods* standing in the mi...,Three *hood* *wearing* *people* stand in a st...,"2,0,4","1,2,3",...,"1,0,2,4,12","0,1,4,7",I

# Prepare Input Sentences

In [53]:
spacy.explain("nsubjpass")

'nominal subject (passive)'

In [66]:
def extract_SVO(txt):
    res = list()
    doc = nlp(txt)

    for cur_nsubj in list(textacy.extract.matches.token_matches(doc, [{"DEP": "nsubj"}])) + list(textacy.extract.matches.token_matches(doc, [{"DEP": "nsubjpass"}])):
        verbs = list()
        if cur_nsubj[0].head.pos_ == "AUX":
            for cur_verb in textacy.extract.matches.token_matches(doc, [{"POS": "VERB"}]):
                if cur_verb[0].head == cur_nsubj[0]:
                    verbs += [cur_verb[0]]
        else:
            verbs += [cur_nsubj[0].head]
        for cur_verb in verbs:
            dobjs = list(textacy.extract.matches.token_matches(doc, [{"DEP": "dobj"}]))
            if len(dobjs) == 0:
                res += [(cur_nsubj[0], cur_verb, None)]
            else:
                for cur_dobj in dobjs:
                    if cur_dobj[0].head == cur_verb:
                        res += [(cur_nsubj[0], cur_verb, cur_dobj[0])]
                    else:
                        res += [(cur_nsubj[0], cur_verb, None)]


    if len(res) == 0:
        cur_root = list(textacy.extract.matches.token_matches(doc, [{"DEP": "ROOT", "POS": "NOUN"}]))
        if len(cur_root) != 0:
            cur_root = cur_root[0][0]
            for cur_verb in textacy.extract.matches.token_matches(doc, [{"DEP": "acl"}]):
                if cur_verb[0].head == cur_root:
                    dobjs = list(textacy.extract.matches.token_matches(doc, [{"DEP": "dobj"}]))
                    if len(dobjs) == 0:
                        res += [(cur_root, cur_verb[0], None)]
                    for cur_dobj in dobjs:
                        if cur_dobj[0].head == cur_verb[0]:
                            res += [(cur_root, cur_verb[0], cur_dobj[0])]
                        else:
                            res += [(cur_root, cur_verb[0], None)]

    return res

In [67]:
gpt_pred['S1_SVO'] = [extract_SVO(gpt_pred.Sentence1.iloc[i]) for i in range(gpt_pred.shape[0])]
gpt_pred['S2_SVO'] = [extract_SVO(gpt_pred.Sentence2.iloc[i]) for i in range(gpt_pred.shape[0])]

In [68]:
gpt_pred[["S1_SVO", "S2_SVO", "Explanation_1"]]

,S1_SVO,S2_SVO,Explanation_1
0,"[(man, walks, bicycle)]","[(man, walks, bike)]","A bicycle and a bike, are the same object."
1,"[(woman, holding, wood), (she, holding, wood)]","[(woman, stands, None)]",If a woman has a smile on her face then she is...
2,"[(dog, jumps, None)]","[(dog, jumping, None)]",Jumping is an action needed to catch a ball.
3,"[(man, wielding, razor), (man, wielding, None)]",[],There cannot be one person present if one man ...
4,"[(group, stares, None), (that, filled, None)]","[(group, analyze, art)]",Not necessary that they analyze the art.
...,...,...,...
995,"[(men, standing, None)]","[(Women, playing, trumpets)]","either there are men on stage, or women playin..."
996,"[(man, standing, None)]","[(people, stand, None)]",3 Man in hoods are hood wearing people.
997,"[(woman, torn, stockings)]","[(man, going, None)]",walking down a street does not mean they are g...
998,"[(men, shining, shoes)]",[],Two men are shining shoes if two men are shini...


In [75]:
has_svo_structure = [not (gpt_pred["S1_SVO"].iloc[i] == list() or gpt_pred["S2_SVO"].iloc[i] == list()) for i in range(gpt_pred.shape[0])]
gpt_pred_svo = gpt_pred[has_svo_structure]

In [76]:
np.mean(has_svo_structure)

0.667

In [77]:
gpt_pred_svo

,Unnamed: 0,pairID,gold_label,Sentence1,Sentence2,Explanation_1,Sentence1_marked_1,Sentence2_marked_1,Sentence1_Highlighted_1,Sentence2_Highlighted_1,...,Explanation_3,Sentence1_marked_3,Sentence2_marked_3,Sentence1_Highlighted_3,Sentence2_Highlighted_3,prompt,pred_explanation,pred_label,S1_SVO,S2_SVO
0,0,4813951931.jpg#3r1e,entailment,A young man in an orange hat walks his bicycle...,A man in an orange hat walks his bike down a s...,"A bicycle and a bike, are the same object.",A young man in an orange hat walks his *bicyc...,A man in an orange hat walks his *bike* down ...,9,8,...,The young man in an orange hart is the man wal...,A *young* *man* in *an* *orange* *hat* *walks...,A *man* in *an* *orange* *hat* *walks* *his* ...,"1,2,4,5,6,7,8,9,10,12","1,5,4,3,11,9,8,7,6",Statement: A man in a suit and dirty dress-sho...,The man is young and the hat is orange.,entailment,"[(man, walks, bicycle)]","[(man, walks, bike)]"
1,1,4678320536.jpg#1r1c,contradiction,A woman standing with smile on her face and sh...,A woman stands holding her baby and crying.,If a woman has a smile on her face then she is...,A woman standing with *smile* *on* *her* *face...,A woman stands holding her baby and *crying.*,"4,5,6,7",7,...,the woman is either crying or has a smile,A *woman* standing with *smile* on her face an...,A *woman* stands holding her baby and *crying.*,"4,1","1,7",Statement: a white dog runs across the water.\...,"The woman is smiling and holding a wood, not a...",contradiction,"[(woman, holding, wood), (she, holding, wood)]","[(woman, stands, None)]"
2,2,661749711.jpg#2r1e,entailment,A dog jumps to catch a red ball outside.,the dog is jumping.,Jumping is an action needed to catch a ball.,A *dog* *jumps* to *catch* a red *ball* *outs...,the *dog* is *jumping.*,"2,7,8,1,4","3,1",...,"If the dog jumps to catch, the dog is jumping.",A dog *jumps* to *catch* a red ball outside.,the dog is *jumping.*,"2,4",3,Statement: A young child rides a bike with a b...,The statement is repeating the same information.,tautology,"[(dog, jumps, None)]","[(dog, jumping, None)]"
4,4,32542645.jpg#0r1n,neutral,A group of people stares at a wall that is fil...,A group of people analyze art together,Not necessary that they analyze the art.,A group of people stares at a wall that is fil...,A group of people *analyze* art together,{},4,...,The group may have just been admiring as oppos...,A group of people stares at a wall that is fil...,A group of people *analyze* art *together*,{},"4,6",Statement: Young mother and child heading some...,Staring at a wall filled with drawings is a fo...,entailment,"[(group, stares, None), (that, filled, None)]","[(group, analyze, art)]"
7,7,4911982235.jpg#0r1c,contradiction,A man in a plaid shirt is walking on a street ...,He is walking on his hands.,The man is either walking or walking on his ha...,A man in a plaid shirt is *walking* on a *stre...,He is *walking* on his *hands.*,"7,10","2,5",...,A man cannot be walking on a street and walkin...,A man in a plaid shirt is *walking* *on* *a* *...,He is *walking* *on* *his* *hands.*,"7,8,9,10","2,3,4,5",Statement: A woman wearing blue pants and a wh...,The statement does not provide any indication ...,contradiction,"[(man, walking, None)]","[(He, walking, None)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,994,3251646144.jpg#0r1n,neutral,A boy is running in a spinning metal tunnel in...,He is getting dizzy.,A boy can run without getting dizzy.,A boy is running in a spinning metal tunnel in...,He is getting *dizzy.*,{},3,...,Running in a spinning metal tunnel does not al...,A boy is running in a spinning metal tunnel in...,He is getting *dizzy.*,{},3,Statement: A man dressed in black is smoking a...,Running in a spinning metal tunnel can cause d...,entailment,"[(boy, running, None)]","[(He, getting, None)]"
995,995,2429284131.jpg#2r1c,contradiction,Three men are standing on stage performing.,Women are playing trumpets on the street.,"ei

# Analyse Predictions

In [4]:
docs = [nlp(test.iloc[i].Explanation_1) for i in range(20)]

In [5]:
for doc in docs:
    for chunk in doc.noun_chunks:
        print(chunk.text, '//', chunk.root.text, '//', chunk.root.dep_, '//', chunk.root.head.text)
    print('-----------------------------------------')

A bicycle // bicycle // nsubj // are
a bike // bike // conj // bicycle
the same object // object // attr // are
-----------------------------------------
a woman // woman // nsubj // has
a smile // smile // dobj // has
her face // face // pobj // on
she // she // nsubj // crying
-----------------------------------------
Jumping // Jumping // nsubj // is
an action // action // attr // is
a ball // ball // dobj // catch
-----------------------------------------
one person // person // attr // be
one man // man // nsubj // shaving
another man's hair // hair // dobj // shaving
-----------------------------------------
they // they // nsubj // analyze
the art // art // dobj // analyze
-----------------------------------------
A snowboarder // snowboarder // nsubj // is
a person // person // attr // is
a snowboarder // snowboarder // nsubj // sails
the air // air // pobj // through
they // they // nsubj // snowboarding
a person // person // nsubj // snowboarding
-----------------------------

In [74]:
doc = nlp(gpt_pred.iloc[9].Sentence2)
html = spacy.displacy.render(doc, style='dep', jupyter=False)
with open('data_vis' + 'test' + '.html', 'w') as f:
    f.write(html)

# Currently Explanation 1 -> Change to pred_explanation

In [112]:
n = gpt_pred.shape[0]
expls = [nlp(gpt_pred.Explanation_1.iloc[i]) for i in range(n)]

In [113]:
verbs_sentence = list()

cur_index = 0
verb_dict = dict()
root_term_dict = dict()

# loop through each row in the test data
for expl in expls:
    cur_verbs = list()
    neg_root = None
    contains_different = False

    # loop through each token in current explanation
    for token in expl:
        if token.lemma_ == 'different':
            contains_different = True
        if token.dep_ == 'ROOT':
            cur_root = token.lemma_
        if token.dep_ == 'neg' and token.head.dep_ == 'ROOT':
            neg_root = token.lemma_ + " " + token.head.lemma_
    if neg_root:
        cur_root = neg_root
    if contains_different:
        cur_root += " different"
    if cur_root not in root_term_dict.keys():
        root_term_dict[cur_root] = [cur_index]
    else:
        root_term_dict[cur_root] += [cur_index]
    cur_index += 1

In [114]:
pd.Series(root_term_dict).apply(lambda x: len(x)).sort_values(ascending=False)

be           254
not be        96
not mean      58
imply         34
not imply     34
            ... 
infer          1
listen         1
lay            1
not write      1
shine          1
Length: 223, dtype: int64

In [122]:
print("Max number of SVO structures in sentence 1: ", gpt_pred_svo.S1_SVO.apply(lambda x: len(x)).max())
print("Max number of SVO structures in sentence 2: ", gpt_pred_svo.S2_SVO.apply(lambda x: len(x)).max())

Max number of SVO structures in sentence 1:  6
Max number of SVO structures in sentence 2:  4


In [138]:
for i in root_term_dict.keys():
    doc = nlp(i)
    print([doc[j] for j in range(len(doc))])

[be]
[not, cry]
[not, be]
[not, necessary]
[not, chew]
[walk]
[be, different]
[constitute]
[help]
[sit]
[have]
[challenge]
[not, infer]
[paper]
[signify]
[not, play]
[not, walk]
[break]
[not, jump]
[not, smile]
[not, mean]
[end]
[imply]
[hold]
[man]
[give]
[not, fall]
[need]
[not, imply]
[not, run]
[not, smoke]
[take]
[push]
[not, tie]
[not, sit]
[complete]
[show]
[ride]
[not, dunk]
[mean]
[yell]
[not, look]
[move]
[go]
[identify]
[swim]
[do]
[fix]
[not, consider]
[stand]
[not, be, different]
[not, limit]
[not, cut]
[race]
[run]
[watch]
[not, have]
[stretch]
[not, stand]
[not, ride]
[bang]
[person]
[not, like]
[not, haul]
[not, work]
[not, block]
[follow]
[contradict]
[fence]
[sound]
[tie]
[not, go]
[find]
[not, transform]
[seat]
[play]
[not, come]
[immerse]
[not, point]
[not, assume]
[not, describe]
[not, mention]
[instal]
[buy]
[wear]
[say]
[toss]
[refer]
[not, specify]
[laugh]
[practice]
[not, do]
[not, surf]
[not, refer]
[call]
[cut]
[not, tell]
[think]
[group]
[jump]
[fell]
[not, 

In [169]:
gpt_pred_svo['reduced_expl'] = None
for root_term in root_term_dict.keys():
    for i in root_term_dict[root_term]:
        try:
            temp = list()
            doc = nlp(gpt_pred_svo.loc[i].Explanation_1)
            for token in doc:
                root_doc = nlp(root_term)
                if token.lemma_ in [root_doc[j].lemma_ for j in range(len(root_doc))]:
                    temp += [token.lemma_]
                for SVO_structure in gpt_pred_svo.loc[i].S1_SVO:
                    if token.lemma_ == SVO_structure[0].lemma_:
                        temp += ["S1_Subj"]
                    if not pd.isna(SVO_structure[1]):
                        if token.lemma_ == SVO_structure[1].lemma_:
                            temp += ["S1_Verb"]
                    if not pd.isna(SVO_structure[2]):
                        if token.lemma_ == SVO_structure[2].lemma_:
                            temp += ["S1_Obj"]
                for SVO_structure in gpt_pred_svo.loc[i].S2_SVO:
                    if token.lemma_ == SVO_structure[0].lemma_:
                        temp += ["S2_Subj"]
                    if not pd.isna(SVO_structure[1]):
                        if token.lemma_ == SVO_structure[1].lemma_:
                            temp += ["S2_Verb"]
                    if not pd.isna(SVO_structure[2]):
                        if token.lemma_ == SVO_structure[2].lemma_:
                            temp += ["S2_Obj"]
            gpt_pred_svo['reduced_expl'].loc[i] = temp
        except:
            None

/var/folders/lp/djryk53n11b_ww_n9xbjm4s80000gn/T/ipykernel_93605/2441747103.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gpt_pred_svo['reduced_expl'] = None
/var/folders/lp/djryk53n11b_ww_n9xbjm4s80000gn/T/ipykernel_93605/2441747103.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gpt_pred_svo['reduced_expl'].loc[i] = temp


In [173]:
gpt_pred_svo[["S1_SVO", "S2_SVO", "Sentence1", "Sentence2", "Explanation_1", "reduced_expl"]]

,S1_SVO,S2_SVO,Sentence1,Sentence2,Explanation_1,reduced_expl
0,"[(man, walks, bicycle)]","[(man, walks, bike)]",A young man in an orange hat walks his bicycle...,A man in an orange hat walks his bike down a s...,"A bicycle and a bike, are the same object.","[S1_Obj, S2_Obj, be]"
1,"[(woman, holding, wood), (she, holding, wood)]","[(woman, stands, None)]",A woman standing with smile on her face and sh...,A woman stands holding her baby and crying.,If a woman has a smile on her face then she is...,"[S1_Subj, S2_Subj, S1_Subj, not, cry]"
2,"[(dog, jumps, None)]","[(dog, jumping, None)]",A dog jumps to catch a red ball outside.,the dog is jumping.,Jumping is an action needed to catch a ball.,[be]
4,"[(group, stares, None), (that, filled, None)]","[(group, analyze, art)]",A group of people stares at a wall that is fil...,A group of people analyze art together,Not necessary that they analyze the art.,"[not, necessary, S1_Subj, S2_Verb, S2_Obj]"
7,"[(man, walking, None)]","[(He, walking, None)]",A man in a plaid shirt is walking on a street ...,He is walking on his hands.,The man is either walking or walking on his ha...,"[S1_Subj, walk, S1_Verb, S2_Verb, walk, S1_Ver..."
...,...,...,...,...,...,...
994,"[(boy, running, None)]","[(He, getting, None)]",A boy is running in a spinning metal tunnel in...,He is getting dizzy.,A boy can run without getting dizzy.,"[S1_Subj, run, S1_Verb, S2_Verb]"
995,"[(men, standing, None)]","[(Women, playing, trumpets)]",Three men are standing on stage performing.,Women are playing trumpets on the street.,"either there are men on stage, or women playin...","[be, S1_Subj, S2_Subj, S2_Verb, S2_Obj]"
996,"[(man, standing, None)]","[(people, stand, None)]",3 young man in hoods standing in the middle of...,Three hood wearing people stand in a street.,3 Man in hoods are hood wearing people.,"[S1_Subj, wear, S2_Subj]"
997,"[(woman, torn, stockings)]","[(man, going, None)]","A woman in skirt, torn stockings and combat bo...",The man and woman are going to a movie in the ...,walking down a street does not mean they are g...,"[not, mean, S2_Verb]"


In [356]:
root_words = list()
n = gpt_pred.shape[0]
cur_index = 0
root_term_dict = dict()
expls = [nlp(gpt_pred.pred_explanation.iloc[i]) for i in range(n)]
for expl in expls:
    for token in expl:
        if token.lemma_ == 'different':
            root_words += [token.head.text]
            if token.head.lemma_ not in root_term_dict.keys():
                root_term_dict[token.head.lemma_] = [cur_index]
            else:
                root_term_dict[token.head.lemma_] += [cur_index]
    cur_index += 1
verbs_sentence = pd.Series(root_words)

In [373]:
for k, v in zip(root_term_dict.keys(), root_term_dict.values()):
    print('=========================')
    print(k)
    try:
        print(gpt_pred.gold_label.iloc[v].value_counts()['contradiction'] / gpt_pred.gold_label.iloc[v].value_counts().sum())
    except:
        print(0)

action
0.9166666666666666
age
0
be
0.7692307692307693
activity
0.88
scene
1.0
setting
0
thing
1.0
scenario
1.0
game
1.0
location
1.0
sport
1.0
term
0
people
1.0
description
1.0
shot
1.0
word
0
item
1.0
background
1.0
time
1.0
environment
0.3333333333333333
same
0
situation
1.0
place
1.0
type
1.0
equipment
1.0
clothing
1.0
animal
1.0
country
0
emotion
1.0
instrument
1.0
dog
1.0
mode
1.0
technique
1.0


In [375]:
gpt_pred.iloc[root_term_dict['action']]

,Unnamed: 0,pairID,gold_label,Sentence1,Sentence2,Explanation_1,Sentence1_marked_1,Sentence2_marked_1,Sentence1_Highlighted_1,Sentence2_Highlighted_1,...,Sentence1_Highlighted_2,Sentence2_Highlighted_2,Explanation_3,Sentence1_marked_3,Sentence2_marked_3,Sentence1_Highlighted_3,Sentence2_Highlighted_3,prompt,pred_explanation,pred_label
8,8,4547257158.jpg#2r1c,contradiction,Four women carrying shoulder bags are walking ...,Four women are running away from a man in the ...,Running and walking are two different gaits.,Four women carrying shoulder bags are *walking...,Four women are *running* away from a man in th...,6,3,...,6,3,walking not running,Four women carrying shoulder bags are *walking...,Four women are *running* away from a man in th...,6,3,"Statement: people playing, basketball ball in ...",The statements are describing two different ac...,contradiction
58,58,4604977910.jpg#1r1c,contradiction,A woman standing in front of a large tank at a...,A woman is tying her shoes,The woman can't be tying her shoes while stand...,A woman *standing* in front of a large tank at...,A woman is *tying* her shoes,2,3,...,"12,13","3,4,5",She cannot be tying her shoes while she turns ...,A woman standing in front of a large tank at a...,A woman is *tying* *her* *shoes*,"12,13","3,4,5",Statement: Man wearing black shirt throwing an...,The statement does not describe the woman stan...,contradiction
153,153,3519155763.jpg#3r1c,contradiction,A dog opens its mouth to catch a blue and yell...,The pet is chewing on a bone,Bones are not blue and yellow.,A dog opens its mouth to catch a *blue* *and* ...,The pet is chewing on a *bone*,"10,11,9,8",6,...,"11,8,10",6,A pet is not necessarily a dog. The pet is ei...,A *dog* *opens* *its* *mouth* *to* *catch* a b...,The *pet* *is* *chewing* on a bone,"1,2,3,4,5,6","1,2,3",Statement: One dog is chasing another dog that...,opening mouth to catch a toy and chewing on a ...,contradiction
216,216,3025315215.jpg#0r1c,contradiction,a brown dog coming out of an orange and yellow...,A brown dog dancing out of a orange and yellow...,The dog cannot be both coming and dancing out ...,a brown *dog* *coming* out of an orange and ye...,A brown *dog* *dancing* out of a orange and ye...,"3,2","3,2",...,3,3,A dog dancing out could not have been coming out.,a brown dog *coming* *out* of an orange and ye...,A brown dog *dancing* *out* of a orange and ye...,"3,4","3,4",Statement: Girl with crossed arms stands in st...,The sentences have different actions (coming o...,contradiction
219,219,3246016210.jpg#4r1c,contradiction,One man cutting through ice on a cold day.,The man is running around his house.,one can not be cutting through ice and be runn...,One man *cutting* *through* *ice* on a cold day.,The man is *running* around his *house.*,"2,3,4","3,6",...,"2,3,4","3,4,5,6",Cutting through ice isnt running around the ho...,One man *cutting* *through* *ice* on a cold day.,The man is *running* *around* his *house.*,"2,3,4","3,4,6",Statement: Two women in their karate uniforms ...,Cutting through ice and running around a house...,contradiction
364,364,7249180494.jpg#0r1c,contradiction,A professional swimmer spits water out after s...,The swimmer is eating a toasted marshmallow.,The swimmer is either swimming or eating a mar...,A professional swimmer spits water out after *...,The swimmer is *eating* a toasted marshmallow.,7,3,...,3,3,"If one spits water, they can not be eating a t...",A professional swimmer *spits* *water* out aft...,The swimmer is *eating* *a* *toasted* *marshma...,"3,4","4,3,5,6",Statement: These two ladies are working in the...,Spitting water out and eating a toasted marshm...,contradiction
385,385,3836884647.jpg#3r1c,contradiction,A man walks by a building at night.,A man is sitting in the sun.,The man cannot simultaneously be sitting and walk,A man *walks* by a building at night.,A man is *sitting* in the sun.,2,3,...,"2,7","3,6",A MAN EITHER WALKS OR HE WILL BE SITTING IN TH...,A man *walks* by a building at

In [405]:
gpt_pred.iloc[608].pred_explanation

'The woman is wearing dark brown leather jacket, sunglasses, brown leather boots, and a silver designer handbag, which implies that she is nicely dressed.'